In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# Menampilkan semua baris dan kolom 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Baca file dataset
df_uji = pd.read_csv("dataset_uji_fuzzy.csv")

# Fungsi keanggotaan segitiga
def triangular(x, a, b, c):
    return np.where(x < a, 0,
           np.where(x <= b, 1 if a == b else (x - a) / (b - a),
           np.where(x <= c, 1 if b == c else (c - x) / (c - b), 0)))

# Fuzzy sets
suhu_luar_sets = {'Sejuk': (10, 10, 24), 'Normal': (22, 27.5, 33), 'Panas': (32, 40, 40)}
suhu_dalam_sets = {'Dingin': (10, 10, 24), 'Normal': (22, 26.5, 31), 'Hangat': (30, 40, 40)}
jumlah_orang_sets = {'Sedikit': (0, 0, 18), 'Sedang': (12, 23, 34), 'Banyak': (30, 50, 50)}

# Fuzzifikasi
def fuzzify_inputs(sl, sd, jo):
    return {
        'suhu_luar': {k: triangular(sl, *v).item() for k, v in suhu_luar_sets.items()},
        'suhu_dalam': {k: triangular(sd, *v).item() for k, v in suhu_dalam_sets.items()},
        'jumlah_orang': {k: triangular(jo, *v).item() for k, v in jumlah_orang_sets.items()}
    }

# Output singleton Sugeno
output_values = {
    'Dingin': 16,
    'Cukup Dingin': 18,
    'Normal': 20,
    'Cukup Hangat': 22,
    'Hangat': 24
}

# Rule base
rules = [
    ('Sejuk', 'Dingin', 'Sedikit', 'Cukup Dingin'),
    ('Sejuk', 'Dingin', 'Sedang', 'Normal'),
    ('Sejuk', 'Dingin', 'Banyak', 'Cukup Hangat'),  
    ('Sejuk', 'Normal', 'Sedikit', 'Normal'),
    ('Sejuk', 'Normal', 'Sedang', 'Cukup Hangat'),  
    ('Sejuk', 'Normal', 'Banyak', 'Hangat'),
    ('Sejuk', 'Hangat', 'Sedikit', 'Cukup Hangat'), 
    ('Sejuk', 'Hangat', 'Sedang', 'Hangat'),
    ('Sejuk', 'Hangat', 'Banyak', 'Hangat'),        
    ('Normal', 'Dingin', 'Sedikit', 'Normal'),
    ('Normal', 'Dingin', 'Sedang', 'Cukup Hangat'), 
    ('Normal', 'Dingin', 'Banyak', 'Hangat'),
    ('Normal', 'Normal', 'Sedikit', 'Cukup Hangat'),
    ('Normal', 'Normal', 'Sedang', 'Hangat'),
    ('Normal', 'Normal', 'Banyak', 'Hangat'),       
    ('Normal', 'Hangat', 'Sedikit', 'Hangat'),
    ('Normal', 'Hangat', 'Sedang', 'Hangat'),       
    ('Normal', 'Hangat', 'Banyak', 'Hangat'),
    ('Panas', 'Dingin', 'Sedikit', 'Cukup Dingin'), 
    ('Panas', 'Dingin', 'Sedang', 'Dingin'),
    ('Panas', 'Dingin', 'Banyak', 'Dingin'),        
    ('Panas', 'Normal', 'Sedikit', 'Cukup Dingin'),
    ('Panas', 'Normal', 'Sedang', 'Dingin'),        
    ('Panas', 'Normal', 'Banyak', 'Dingin'),
    ('Panas', 'Hangat', 'Sedikit', 'Dingin'),       
    ('Panas', 'Hangat', 'Sedang', 'Dingin'),
    ('Panas', 'Hangat', 'Banyak', 'Dingin'),
]

# Inferensi Sugeno
def sugeno_inference(sl, sd, jo):
    fuzzy = fuzzify_inputs(sl, sd, jo)
    numerator, denominator = 0, 0
    for sl_cat, sd_cat, jo_cat, output_cat in rules:
        alpha = min(fuzzy['suhu_luar'][sl_cat], fuzzy['suhu_dalam'][sd_cat], fuzzy['jumlah_orang'][jo_cat])
        z = output_values[output_cat]
        numerator += alpha * z
        denominator += alpha
    return None if denominator == 0 else numerator / denominator

# Proses prediksi seluruh dataset
df_uji['prediksi'] = df_uji.apply(
    lambda row: round(sugeno_inference(row['suhu_luar'], row['suhu_dalam'], row['jumlah_orang']), 1), axis=1
)

# Deteksi kolom suhu aktual
if 'suhu_optimal_aktual' in df_uji.columns:
    df_uji['suhu_aktual'] = df_uji['suhu_optimal_aktual']
else:
    alt_cols = ['actual_temp', 'suhu_sebenarnya', 'target', 'actual', 'suhu_target']
    found = False
    for col in alt_cols:
        if col in df_uji.columns:
            df_uji['suhu_aktual'] = df_uji[col]
            found = True
            print(f"Kolom aktual ditemukan: '{col}'")
            break
    if not found:
        print("Kolom suhu aktual tidak ditemukan. Gunakan prediksi sebagai placeholder.")
        df_uji['suhu_aktual'] = df_uji['prediksi']

# Konversi ke kelas suhu: <=20 (kelas 0), >20 (kelas 1)
df_uji['aktual_kelas'] = df_uji['suhu_aktual'].apply(lambda x: 0 if x <= 20 else 1)
df_uji['prediksi_kelas'] = df_uji['prediksi'].apply(lambda x: 0 if x <= 20 else 1)

# Evaluasi model
cm = confusion_matrix(df_uji['aktual_kelas'], df_uji['prediksi_kelas'])
tn, fp, fn, tp = cm.ravel()
akurasi = accuracy_score(df_uji['aktual_kelas'], df_uji['prediksi_kelas']) * 100
sensitivitas = tp / (tp + fn) * 100 if (tp + fn) != 0 else 0

print("\n=== HASIL EVALUASI FUZZY SUGENO ===")
print(f"Akurasi      : {akurasi:.2f}%")
print(f"Sensitivitas : {sensitivitas:.2f}%")

# Tampilkan semua hasil prediksi
print("\n=== HASIL PREDIKSI LENGKAP ===")
print(df_uji[['suhu_luar', 'suhu_dalam', 'jumlah_orang', 'suhu_aktual', 'prediksi']])

# Simpan hasil prediksi ke file CSV
df_uji.to_csv("hasil_prediksi_sugeno.csv", index=False)
print("\nHasil prediksi telah disimpan ke 'hasil_prediksi_sugeno.csv'")


=== HASIL EVALUASI FUZZY SUGENO ===
Akurasi      : 76.15%
Sensitivitas : 80.82%

=== HASIL PREDIKSI LENGKAP ===
     suhu_luar  suhu_dalam  jumlah_orang  suhu_aktual  prediksi
0         22.0        23.5            10         19.5      19.6
1         30.0        20.0            15         22.0      21.2
2         17.0        19.5            25         21.0      20.0
3         20.5        25.0            12         20.0      20.0
4         34.5        20.0            30         17.0      16.0
5         36.0        29.0            35         15.5      16.0
6         32.0        22.5            18         22.0      23.0
7         33.0        31.0            40         15.0      16.0
8         28.0        23.0            27         21.5      23.5
9         26.5        24.0            12         22.0      22.0
10        21.5        28.0            16         20.0      21.2
11        25.0        21.0            14         16.5      20.9
12        17.0        26.5            35         17.0  